# Trong đại dịch Covid-19 có nguồn gốc từ Wuhan, Trung Quốc đã làm ảnh hưởng tới cuộc sống của nhân loại, cướp đi sinh mạng của ít nhất 200.000 người vô tội và sẽ còn tiếp tục tăng trong thời gian tới.
## Để phục vụ công tác chẩn đoán bệnh, các nhà khoa học đã tìm cách áp dụng trí thông minh nhân tạo vào trong việc xử lí và chẩn đoán ảnh CT và X quang chụp phổi để đánh giá tổn thương và phân loại viêm phổi do các nguyên nhân khác nhau, trong đó có Covid-19.
## Trong bài này, mình sử dụng dataset tại đây: https://covidresearch.ai/datasets/dataset?id=2. Theo như tìm hiểu về cơ sở dữ liệu này, có lẽ nó được tiếp thu từ 2 nghiên cứu trước đó là bài báo này https://arxiv.org/abs/2003.11597 (địa chỉ github: https://github.com/ieee8023/covid-chestxray-dataset) và bài báo này https://arxiv.org/abs/2003.09871 (https://github.com/lindawangg/COVID-Net).
## Gần đây có bài báo công bố sử dụng mạng EfficientNet (bài báo về mạng tại đây https://arxiv.org/pdf/1905.11946.pdf) để chẩn đoán dataset này cho kết quả có độ nhạy và độ đặc hiệu cao hơn hẳn các kết quả trước đó. Các bạn có thể tham khảo bài báo này tại đây: https://arxiv.org/pdf/2004.05717.pdf. Kết quả bài báo chỉ ra rằng họ đã thêm vào mạng EfficientNet_B0 một số lớp để cải thiện khả năng phân loại. Tuy nhiên bài báo sử dụng Framework là Keras, còn trong bài lặp lại thí nghiệm này, mình sử dụng Framework là PyTorch với đóng góp rất lớn của anh Ross Wightman khi xây dựng các mạng thần kinh tích chập sâu cho công việc phân loại ảnh (các bạn có thể tham khảo code tại đây https://github.com/rwightman/pytorch-image-models).
### Bên cạnh việc sử dụng Framework khác với bài báo gốc, mình cũng có 1 số thay đổi như mình dùng hàm tối ưu là SGD thay vì ADAM, và mình bổ thêm kĩ thuật Augmentation (ở đây mình dùng thêm kĩ thuật RandAugmentation tại bài báo này https://arxiv.org/abs/1909.13719) để nâng cao độ chính xác.
### Mình cũng đã thử huấn luyện dataset này với các mạng khác nhau, tuy nhiên kết quả phân loại có lẽ vẫn hiệu quả nhất với mạng EfficientNet_B0.
### Tuy nhiên, để mô hình này có thể sử dụng trong thực tiễn, chắc chắn cần phải tiến hành internal và external validity qua nhiều bước khác nhau. Thêm vào đó, chúng ta hoàn toàn có thể nghĩ đến kĩ thuật ensemble voting để tăng tính chính xác cho công cụ chẩn đoán!
# For fun, mình xây dựng thử nền tảng web dùng cho chẩn đoán các ảnh X quang vùng ngực xem bệnh nhân có nhiễm Covid-19 hay không. Các bạn có thể tham khảo tại địa chỉ github của minh [https://github.com/linhduongtuan/Covid-19_Xray_Classifier/]

In [1]:
import os
import re
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import logging
import fnmatch
import argparse
import torchvision
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
from sklearn import metrics
import torch.optim as optim
from datetime import datetime
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
import torch.utils.model_zoo as model_zoo
from timm.models.layers.activations import *
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict, defaultdict
from torchvision import transforms, models, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from randaugment import RandAugment, ImageNetPolicy, Cutout
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [2]:
data_dir = '/home/linh/Downloads/Covid-19_CT'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/test'

# Define your transforms for the training and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        RandAugment(),
        ImageNetPolicy(),
        Cutout(size=16),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
batch_size = 44
data_loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4, pin_memory = True)
              for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes
print(class_names)
print(dataset_sizes)
print(device)

### we get the class_to_index in the data_Set but what we really need is the cat_to_names  so we will create
_ = image_datasets['train'].class_to_idx
cat_to_name = {_[i]: i for i in list(_.keys())}
print(cat_to_name)
    
# Run this to test the data loader
images, labels = next(iter(data_loader['test']))
images.size()

['COVID', 'non-COVID']
{'train': 1985, 'test': 497}
cuda:0
{0: 'COVID', 1: 'non-COVID'}


torch.Size([44, 3, 224, 224])

In [3]:
"""def showimage(data_loader, number_images, cat_to_name):
    dataiter = iter(data_loader)
    images, labels = dataiter.next()
    images = images.numpy() # convert images to numpy for display
    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(number_images, 4))
    for idx in np.arange(number_images):
        ax = fig.add_subplot(2, number_images/2, idx+1, xticks=[], yticks=[])
        img = np.transpose(images[idx])
        plt.imshow(img)
        ax.set_title(cat_to_name[labels.tolist()[idx]])
        
#### to show some  images
showimage(data_loader['test'], 20, cat_to_name)"""

"def showimage(data_loader, number_images, cat_to_name):\n    dataiter = iter(data_loader)\n    images, labels = dataiter.next()\n    images = images.numpy() # convert images to numpy for display\n    # plot the images in the batch, along with the corresponding labels\n    fig = plt.figure(figsize=(number_images, 4))\n    for idx in np.arange(number_images):\n        ax = fig.add_subplot(2, number_images/2, idx+1, xticks=[], yticks=[])\n        img = np.transpose(images[idx])\n        plt.imshow(img)\n        ax.set_title(cat_to_name[labels.tolist()[idx]])\n        \n#### to show some  images\nshowimage(data_loader['test'], 20, cat_to_name)"

In [4]:
#model = models.resnet50(pretrained=True)
#model = timm.create_model('resnet50', pretrained=True)
model = timm.create_model('mixnet_xl', pretrained=True)
#model.fc #show fully connected layer for ResNet family
model.classifier #show the classifier layer (fully connected layer) for EfficientNets

Linear(in_features=1536, out_features=1000, bias=True)

In [5]:
# Create classifier
for param in model.parameters():
    param.requires_grad = True
# define `classifier` for ResNet
# Otherwise, define `fc` for EfficientNet family 
#because the definition of the full connection/classifier of 2 CNN families is differnt
fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(1536, 1000, bias=True)),
							     ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								 ('dropout1', nn.Dropout(0.7)),
                                 ('fc2', nn.Linear(1000, 512)),
								 ('BN2', nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								 ('swish1', Swish()),
								 ('dropout2', nn.Dropout(0.5)),
								 ('fc3', nn.Linear(512, 128)),
								 ('BN3', nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
							     ('swish2', Swish()),
								 ('fc4', nn.Linear(128, 2)),
								 ('output', nn.Softmax(dim=1))
							 ]))
# connect base model (EfficientNet_B0) with modified classifier layer
model.fc = fc
criterion = nn.CrossEntropyLoss()
#optimizer = Nadam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
optimizer = optim.SGD(model.parameters(), 
                      lr=0.01,momentum=0.9,
                      nesterov=True,
                      weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
#show our model architechture and send to GPU
model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(model)
print("The number of parameters of the model is:", count)

The number of parameters of the model is: 14015482


In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=200, checkpoint = None):
    since = time.time()

    if checkpoint is None:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = math.inf
        best_acc = 0.
    else:
        print(f'Val loss: {checkpoint["best_val_loss"]}, Val accuracy: {checkpoint["best_val_accuracy"]}')
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        best_loss = checkpoint['best_val_loss']
        best_acc = checkpoint['best_val_accuracy']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, (inputs, labels) in enumerate(data_loader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                if i % 1000 == 999:
                    print('[%d, %d] loss: %.8f' % 
                          (epoch + 1, i, running_loss / (i * inputs.size(0))))

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':                
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':                
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.8f} Acc: {:.8f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_loss < best_loss:
                print(f'New best model found!')
                print(f'New record loss: {epoch_loss}, previous record loss: {best_loss}')
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'best_val_loss': best_loss,
                            'best_val_accuracy': best_acc,
                            'scheduler_state_dict' : scheduler.state_dict(),
                            }, 
                            CHECK_POINT_PATH
                            )
                print(f'New record loss is SAVED: {epoch_loss}')

        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.8f} Best val loss: {:.8f}'.format(best_acc, best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss, best_acc

In [ ]:
CHECK_POINT_PATH = '/home/linh/Downloads/Covid-19_CT/weights/MixNet_Extra_Large_Covid-19.pth'
try:
    checkpoint = torch.load(CHECK_POINT_PATH)
    print("checkpoint loaded")
except:
    checkpoint = None
    print("checkpoint not found")
if checkpoint == None:
    CHECK_POINT_PATH = CHECK_POINT_PATH
model, best_val_loss, best_val_acc = train_model(model,
                                                 criterion,
                                                 optimizer,
                                                 scheduler,
                                                 num_epochs = 200,
                                                 checkpoint = torch.load(CHECK_POINT_PATH)
                                                 ) 
                                                
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss,
            'best_val_accuracy': best_val_acc,
            'scheduler_state_dict': scheduler.state_dict(),
            }, CHECK_POINT_PATH)

checkpoint loaded
Val loss: 0.02006731217196257, Val accuracy: 0.9919517102615695
Epoch 0/199
----------
train Loss: 0.10405291 Acc: 0.95465995
test Loss: 0.02198312 Acc: 0.99195171

Epoch 1/199
----------
train Loss: 0.11100209 Acc: 0.95516373
test Loss: 0.02469608 Acc: 0.99195171

Epoch 2/199
----------
train Loss: 0.10546896 Acc: 0.95617128
test Loss: 0.02382426 Acc: 0.98591549

Epoch 3/199
----------
train Loss: 0.09845995 Acc: 0.96322418
test Loss: 0.02107511 Acc: 0.98993964

Epoch 4/199
----------
train Loss: 0.10399646 Acc: 0.95919395
test Loss: 0.02335610 Acc: 0.98993964

Epoch 5/199
----------
train Loss: 0.11653365 Acc: 0.94811083
test Loss: 0.02213998 Acc: 0.98993964

Epoch 6/199
----------
train Loss: 0.10562727 Acc: 0.95163728
test Loss: 0.01988330 Acc: 0.99195171
New best model found!
New record loss: 0.0198832995078415, previous record loss: 0.02006731217196257
New record loss is SAVED: 0.0198832995078415

Epoch 7/199
----------
train Loss: 0.11050649 Acc: 0.95264484
tes